In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Flight Data")

In [3]:
!ls

2016_original.zip  2017_original.zip


In [4]:
import zipfile
import os

# Google Colab local cache directory (usually '/content')
local_cache_dir = '/content'

# Path to the zip files in Google Drive
drive_zip_files = ['/content/drive/MyDrive/Flight Data/2016_original.zip',
                   '/content/drive/MyDrive/Flight Data/2017_original.zip']

# Extracting the zip files to the local cache of Google Colab
for zip_file in drive_zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        # Extracting the contents to the local cache directory
        zip_ref.extractall(local_cache_dir)

In [5]:
import os
os.chdir("/content/drive/MyDrive/weather")
!ls

ATL  CLT  DEN  DFW  EWR  IAH  JFK  LAS	LAX  MCO  MIA  ORD  PHX  SEA  SFO


# FLIGHT 2016 year-1 Month data

In [6]:
import pandas as pd

df = pd.read_csv("/content/2016/On_Time_On_Time_Performance_2016_1/On_Time_On_Time_Performance_2016_1.csv")
df

<ipython-input-6-cb150fd1c750>:3: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/2016/On_Time_On_Time_Performance_2016_1/On_Time_On_Time_Performance_2016_1.csv")


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445822,2016,1,1,3,7,2016-01-03,F9,20436,F9,N934FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445823,2016,1,1,3,7,2016-01-03,F9,20436,F9,N934FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445824,2016,1,1,3,7,2016-01-03,F9,20436,F9,N935FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445825,2016,1,1,3,7,2016-01-03,F9,20436,F9,N935FR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Merging for Just SFO airport of 2017-1**

In [7]:
import pandas as pd
import os
import json

# Define the path to the weather data directory and list of airport codes
weather_data_dir = '/content/drive/MyDrive/weather'
airport_codes = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]

# Recommended columns for weather and flight data
weather_columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM', 'Visibility', 'Pressure', 'Cloudcover', 'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time', 'airport']
flight_columns = ['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'Origin', 'Dest', 'ArrTime', 'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes']

# Load the flight data
flight_data = pd.read_csv('/content/2017/On_Time_On_Time_Performance_2017_1/On_Time_On_Time_Performance_2017_1.csv')

# Function to process weather data for a given airport
def process_weather_data(airport_code):
    # Construct the file path for the JSON file
    file_path = os.path.join(weather_data_dir, airport_code, '2017-1.json')
    if not os.path.exists(file_path):
        return None

    # Load the JSON data
    with open(file_path, 'r') as file:
        weather_data = json.load(file)

    # Normalize the nested data
    weather_records = []
    for daily_data in weather_data['data']['weather']:
        date = daily_data['date']
        for hourly_data in daily_data['hourly']:
            hourly_data['date'] = date  # Add date to each hourly record
            weather_records.append(hourly_data)

    weather_df = pd.DataFrame(weather_records)

    # Select required columns and rename them
    weather_df = weather_df[['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
                             'visibility', 'pressure', 'cloudcover', 'DewPointF',
                             'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date', 'time']]
    weather_df.columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM',
                          'Visibility', 'Pressure', 'Cloudcover', 'DewPointF',
                          'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time']
    weather_df['airport'] = airport_code

    return weather_df

# Process weather data for each airport and merge with flight data
all_merged_data = pd.DataFrame()

for code in airport_codes:
    weather_df = process_weather_data(code)
    if weather_df is not None:
        # Filter flight data for the current airport and select required columns
        filtered_flight_data = flight_data[(flight_data['Origin'] == code) | (flight_data['Dest'] == code)]
        filtered_flight_data = filtered_flight_data[flight_columns]

        # Merge the weather data with the flight data
        merged_data = pd.merge(filtered_flight_data, weather_df, left_on=['FlightDate', 'Origin'], right_on=['date', 'airport'], how='left')
        all_merged_data = pd.concat([all_merged_data, merged_data], ignore_index=True)

<ipython-input-7-290fe3341a25>:14: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  flight_data = pd.read_csv('/content/2017/On_Time_On_Time_Performance_2017_1/On_Time_On_Time_Performance_2017_1.csv')


In [8]:
all_merged_data

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,...,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,date,time,airport
0,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,38,23,43,38,83,2017-01-01,0,ATL
1,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,39,22,44,38,85,2017-01-01,100,ATL
2,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,40,21,44,39,87,2017-01-01,200,ATL
3,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,41,21,44,39,89,2017-01-01,300,ATL
4,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,42,21,45,40,91,2017-01-01,400,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038172,2017-01-23,1,2017,1,23,1433.0,1.0,1350,43.0,STL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5038173,2017-01-24,1,2017,1,24,1650.0,0.0,1655,0.0,BUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5038174,2017-01-24,1,2017,1,24,1319.0,0.0,1330,0.0,BUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5038175,2017-01-24,1,2017,1,24,2002.0,0.0,2005,0.0,BUR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
all_merged_data.head(5)

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,...,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,date,time,airport
0,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,38,23,43,38,83,2017-01-01,0,ATL
1,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,39,22,44,38,85,2017-01-01,100,ATL
2,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,40,21,44,39,87,2017-01-01,200,ATL
3,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,41,21,44,39,89,2017-01-01,300,ATL
4,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,42,21,45,40,91,2017-01-01,400,ATL


In [10]:
pwd

'/content/drive/.shortcut-targets-by-id/1FH3SzcDlcDVy4QkwB7z4VNi1bE18d0JA/weather'

In [11]:
# Change the current working directory to a location where you have write permissions
os.chdir('/content')

# Save the merged data to 'merged_data.csv'
all_merged_data.to_csv('merged_data.csv', index=False)

**binary classification problem**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the target variable and features
target_variable = 'ArrDelayMinutes'
features = ['Month', 'DayofMonth', 'DepTime', 'CRSDepTime', 'DepDelayMinutes',
            'WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM',
            'Visibility', 'Pressure', 'Cloudcover', 'DewPointF',
            'WindGustKmph', 'tempF', 'WindChillF', 'Humidity']

# Drop rows with missing values in target variable or features
all_merged_data = all_merged_data.dropna(subset=[target_variable] + features)

# Create binary target variable (1 for delayed, 0 for not delayed)
all_merged_data['Delayed'] = (all_merged_data[target_variable] > 0).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_merged_data[features], all_merged_data['Delayed'], test_size=0.2, random_state=42)

# Train Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

# Predictions on the test set
dt_predictions = dt_classifier.predict(X_test)

# Evaluate Decision Tree Classifier
print("Decision Tree Classifier Performance:")
print("Accuracy:", accuracy_score(y_test, dt_predictions))
print("Classification Report:\n", classification_report(y_test, dt_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, dt_predictions))

# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Predictions on the test set
rf_predictions = rf_classifier.predict(X_test)

# Evaluate Random Forest Classifier
print("\nRandom Forest Classifier Performance:")
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print("Classification Report:\n", classification_report(y_test, rf_predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_predictions))


<ipython-input-14-fcc0ee24fc4a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_merged_data['Delayed'] = (all_merged_data[target_variable] > 0).astype(int)


Decision Tree Classifier Performance:
Accuracy: 0.9549751603469002
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96    555508
           1       0.95      0.94      0.95    391969

    accuracy                           0.95    947477
   macro avg       0.95      0.95      0.95    947477
weighted avg       0.95      0.95      0.95    947477

Confusion Matrix:
 [[536248  19260]
 [ 23400 368569]]


# **Final Code**

In [ ]:
import pandas as pd
import os
import json

# Define the path to the weather data directory and list of airport codes
weather_data_dir = '/content/drive/MyDrive/weather'
airport_codes = ["ATL", "CLT", "DEN", "DFW", "EWR", "IAH", "JFK", "LAS", "LAX", "MCO", "MIA", "ORD", "PHX", "SEA", "SFO"]

# Recommended columns for weather and flight data
weather_columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM', 'Visibility', 'Pressure', 'Cloudcover', 'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time', 'airport']
flight_columns = ['FlightDate', 'Quarter', 'Year', 'Month', 'DayofMonth', 'DepTime', 'DepDel15', 'CRSDepTime', 'DepDelayMinutes', 'Origin', 'Dest', 'ArrTime', 'CRSArrTime', 'ArrDel15', 'ArrDelayMinutes']

# Function to process weather data for a given airport
def process_weather_data(airport_code, year, month):
    # Construct the file path for the JSON file
    file_path = os.path.join(weather_data_dir, airport_code, f'{year}-{month}.json')
    if not os.path.exists(file_path):
        return None

    # Load the JSON data
    with open(file_path, 'r') as file:
        weather_data = json.load(file)

    # Normalize the nested data
    weather_records = []
    for daily_data in weather_data['data']['weather']:
        date = daily_data['date']
        for hourly_data in daily_data['hourly']:
            hourly_data['date'] = date  # Add date to each hourly record
            weather_records.append(hourly_data)

    weather_df = pd.DataFrame(weather_records)

    # Select required columns and rename them
    weather_df = weather_df[['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
                             'visibility', 'pressure', 'cloudcover', 'DewPointF',
                             'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date', 'time']]
    weather_df.columns = ['WindSpeedKmph', 'WindDirDegree', 'WeatherCode', 'precipMM',
                          'Visibility', 'Pressure', 'Cloudcover', 'DewPointF',
                          'WindGustKmph', 'tempF', 'WindChillF', 'Humidity', 'date', 'time']
    weather_df['airport'] = airport_code

    return weather_df

# Main loop to process and merge data for each year and month
all_merged_data = pd.DataFrame()
years = [2016,2017]
months = range(1, 13)

for year in years:
    for month in months:
        # Load the flight data for the current year and month
        flight_data_file = f'/content/{year}/On_Time_On_Time_Performance_{year}_{month}/On_Time_On_Time_Performance_{year}_{month}.csv'
        if not os.path.exists(flight_data_file):
            continue
        flight_data = pd.read_csv(flight_data_file, usecols=flight_columns)

        for code in airport_codes:
            weather_df = process_weather_data(code, year, month)
            if weather_df is not None:
                # Filter flight data for the current airport and select required columns
                filtered_flight_data = flight_data[(flight_data['Origin'] == code) | (flight_data['Dest'] == code)]
                filtered_flight_data = filtered_flight_data[flight_columns]

                # Merge the weather data with the flight data
                merged_data = pd.merge(filtered_flight_data, weather_df, left_on=['FlightDate', 'Origin'], right_on=['date', 'airport'], how='left')
                all_merged_data = pd.concat([all_merged_data, merged_data], ignore_index=True)

# The above is the code for merging the entire flights airport to corresponding weather details. **Due to lack of required RAM in colab, it is crashing**.

In [ ]:
all_merged_data.shape

In [ ]:
all_merged_data.head(5)

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,...,Pressure,Cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,Humidity,date,time,airport
0,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,38,23,43,38,83,2017-01-01,0,ATL
1,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,39,22,44,38,85,2017-01-01,100,ATL
2,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1020,100,40,21,44,39,87,2017-01-01,200,ATL
3,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,41,21,44,39,89,2017-01-01,300,ATL
4,2017-01-01,1,2017,1,1,1124.0,0.0,1130,0.0,ATL,...,1019,100,42,21,45,40,91,2017-01-01,400,ATL
